In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName('USER_ROW_TO_TRUSTED') \
    .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension') \
    .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog') \
    .getOrCreate()

In [5]:
spark

In [6]:
raw = 's3a://raw/users'
trusted = 's3a://trusted/users'

In [7]:
df_raw = spark.read.json(raw)

In [10]:
df_filter = df_raw.select('id', 'first_name', 'last_name', 'username', 'email')

In [11]:
df_filter.show()

+----+----------+---------+-----------------+--------------------+
|  id|first_name|last_name|         username|               email|
+----+----------+---------+-----------------+--------------------+
|4127|   Marlana|Abernathy|marlana.abernathy|marlana.abernathy...|
|8081|  Epifania|    Koepp|   epifania.koepp|epifania.koepp@em...|
|3631|     Tyler|    Crist|      tyler.crist|tyler.crist@email...|
|6918|   Hershel|    Berge|    hershel.berge|hershel.berge@ema...|
|9805|    Luther|     Dare|      luther.dare|luther.dare@email...|
|9034|      Beau|   Sawayn|      beau.sawayn|beau.sawayn@email...|
|6668|      Noel|   Hirthe|      noel.hirthe|noel.hirthe@email...|
+----+----------+---------+-----------------+--------------------+



In [12]:
from delta.tables import DeltaTable

In [14]:
if not DeltaTable.isDeltaTable(spark, trusted):
    df_filter.write.format('delta').mode('append').save(trusted)

In [15]:
delta_table  = DeltaTable.forPath(spark, trusted)

In [16]:
delta_table.alias('trusted') \
    .merge(df_filter.alias('raw'), 'trusted.id = raw.id') \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()
        

In [17]:
df_parquet = spark.read.parquet(trusted)

In [18]:
df_parquet.count()

14

In [19]:
dt = DeltaTable.forPath(spark, trusted)

In [20]:
dt.toDF().count()

7

In [21]:
df_teste = spark.read.parquet('s3a://trusted/users/part-00000-919f7f90-1193-4a5b-b1b8-e13d9994e5f7-c000.snappy.parquet')

In [23]:
df_teste.count()

7